

*   Author: **Sevdenaz Yılmaz**
*   ID: **30300**
*   Date: **14.11.2024**






Required Imports and Basic Evaluation Function:

In [1]:
import random
import time
import math

def evaluate(state):
    """Evaluate the number of conflicts in a given state."""
    conflicts = 0
    n = len(state)
    for i in range(n):
        for j in range(i + 1, n):
            if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                conflicts += 1
    return conflicts

Neighbor State Generation Helper Function:

In [3]:
def get_neighbors(current):
    """Generate all neighboring states by moving one queen at a time."""
    neighbors = []
    n = len(current)
    for i in range(n):
        for j in range(n):
            if current[i] != j:
                neighbor = current[:]
                neighbor[i] = j
                neighbors.append(neighbor)
    return neighbors

Basic Hill Climbing Algorithm:

In [4]:
def hill_climbing(initial_state):
    """Basic hill climbing algorithm."""
    current = initial_state
    current_eval = evaluate(current)
    iterations = 0

    while iterations < 200:  # Increased for N=20
        neighbors = get_neighbors(current)
        next_state = min(neighbors, key=evaluate)
        next_eval = evaluate(next_state)

        if next_eval >= current_eval:
            break

        current = next_state
        current_eval = next_eval
        iterations += 1

    return current, iterations

Random Restart Hill Climbing Implementation:

In [5]:
def random_restart_hill_climbing(n, k):
    """Hill climbing with k random restarts."""
    best_state = None
    best_score = float('inf')
    total_iterations = 0

    for _ in range(k):
        initial_state = [random.randint(0, n - 1) for _ in range(n)]
        result, iters = hill_climbing(initial_state)
        result_score = evaluate(result)
        total_iterations += iters

        if result_score < best_score:
            best_score = result_score
            best_state = result

        if best_score == 0:  # Early termination if solution found
            break

    return best_state, total_iterations

Random Neighbor and Stochastic Hill Climbing:

In [6]:
def random_neighbor(current_state):
    """Generate a random neighbor by moving one queen randomly."""
    neighbor = current_state[:]
    i = random.randint(0, len(current_state) - 1)
    possible_positions = list(range(len(current_state)))
    possible_positions.remove(current_state[i])
    if possible_positions:
        new_position = random.choice(possible_positions)
        neighbor[i] = new_position
    return neighbor

def stochastic_hill_climbing(initial_state):
    """Stochastic hill climbing algorithm."""
    current = initial_state
    current_eval = evaluate(current)
    iterations = 0
    max_attempts = len(initial_state) * 2  # Increased for N=20

    while iterations < 1000:  # Increased for N=20
        better_found = False
        attempts = 0

        while attempts < max_attempts:
            neighbor = random_neighbor(current)
            neighbor_eval = evaluate(neighbor)

            if neighbor_eval < current_eval:
                current = neighbor
                current_eval = neighbor_eval
                better_found = True
                break
            attempts += 1

        if not better_found or current_eval == 0:
            break
        iterations += 1

    return current, iterations

Simulated Annealing Implementation:

In [7]:
def simulatedAnnealing(board, maxIter=1000):
    """
    Implement simulated annealing for N-queens problem
    Initial temperature T = 10000, cooling rate α = 0.95
    """
    import random
    import math

    T = 10000  # Initial temperature
    alpha = 0.95  # Cooling rate
    n = len(board)
    current = board.copy()
    current_cost = calculateCost(current)

    for _ in range(maxIter):
        T = T * alpha  # Cool down
        if T < 0.01:  # Stop if temperature is too low
            break

        # Generate a neighbor by randomly moving one queen
        new_board = current.copy()
        i = random.randint(0, n-1)  # Choose random queen
        new_pos = random.randint(0, n-1)  # Choose new position
        new_board[i] = new_pos

        new_cost = calculateCost(new_board)

        # Calculate change in cost
        delta_E = new_cost - current_cost

        # Accept if better or with probability e^(-delta_E/T)
        if delta_E < 0 or random.random() < math.exp(-delta_E / T):
            current = new_board
            current_cost = new_cost

        if current_cost == 0:  # Solution found
            return current, True

    return current, current_cost == 0

Experiment Running and Results Collection:

In [43]:
def run_experiments(n=20, trials=100):
    print(f"\nRunning experiments with N={n}, Trials={trials}")
    results = {}

    # Basic Hill Climbing
    print("\nTesting Basic Hill Climbing...")
    successes = 0
    total_time_start = time.time()
    total_iterations = 0

    for i in range(trials):
        if (i+1) % 10 == 0:
            print(f"Completed {i+1}/{trials} trials...")
        board = generate_random_board(n)
        _, success, iters = hill_climbing(board)
        if success:
            successes += 1
            total_iterations += iters

    elapsed_time = time.time() - total_time_start
    results['basic'] = {
        'success_rate': (successes/trials) * 100,
        'time': elapsed_time,
        'avg_iterations': total_iterations/trials if successes > 0 else 0
    }

    # Random Restart (k=10)
    print("\nTesting Random Restart (k=10)...")
    successes = 0
    total_time_start = time.time()
    total_restarts = 0
    total_iterations = 0

    for i in range(trials):
        if (i+1) % 10 == 0:
            print(f"Completed {i+1}/{trials} trials...")
        _, success, restarts, iters = random_restart(n, 10)
        if success:
            successes += 1
            total_restarts += restarts
            total_iterations += iters

    elapsed_time = time.time() - total_time_start
    results['restart10'] = {
        'success_rate': (successes/trials) * 100,
        'time': elapsed_time,
        'avg_restarts': total_restarts/successes if successes > 0 else 0,
        'avg_iterations': total_iterations/trials if successes > 0 else 0
    }

    # Random Restart (k=100)
    print("\nTesting Random Restart (k=100)...")
    successes = 0
    total_time_start = time.time()
    total_restarts = 0
    total_iterations = 0

    for i in range(trials):
        if (i+1) % 10 == 0:
            print(f"Completed {i+1}/{trials} trials...")
        _, success, restarts, iters = random_restart(n, 100)
        if success:
            successes += 1
            total_restarts += restarts
            total_iterations += iters

    elapsed_time = time.time() - total_time_start
    results['restart100'] = {
        'success_rate': (successes/trials) * 100,
        'time': elapsed_time,
        'avg_restarts': total_restarts/successes if successes > 0 else 0,
        'avg_iterations': total_iterations/trials if successes > 0 else 0
    }

    # Stochastic Hill Climbing
    print("\nTesting Stochastic Hill Climbing...")
    successes = 0
    total_time_start = time.time()
    total_iterations = 0

    for i in range(trials):
        if (i+1) % 10 == 0:
            print(f"Completed {i+1}/{trials} trials...")
        board = generate_random_board(n)
        _, success, iters = stochastic_hill_climbing(board)
        if success:
            successes += 1
            total_iterations += iters

    elapsed_time = time.time() - total_time_start
    results['stochastic'] = {
        'success_rate': (successes/trials) * 100,
        'time': elapsed_time,
        'avg_iterations': total_iterations/trials if successes > 0 else 0
    }

# Simulated Annealing
    print("\nTesting Simulated Annealing...")
    successes = 0
    total_time_start = time.time()

    for i in range(trials):
        if (i+1) % 10 == 0:
            print(f"Completed {i+1}/{trials} trials...")
        success = NQueen2(n, simulated=True, temp=10000)  # NQueen2 sadece bool döndüğü için direkt alıyoruz
        if success:
            successes += 1

    elapsed_time = time.time() - total_time_start
    results['simulated_annealing'] = {
        'success_rate': (successes/trials) * 100,
        'time': elapsed_time
    }


    # Print Final Results
    print("\nFinal Results:")
    print("-" * 80)
    print(f"{'Algorithm':<25} {'Success Rate':<15} {'Time (s)':<15} {'Avg Restarts':<15}")
    print("-" * 80)

    print(f"Basic Hill Climbing{' '*8} {results['basic']['success_rate']:>6.1f}% {results['basic']['time']:>14.3f} {'-':>14}")
    print(f"Random Restart (k=10){' '*4} {results['restart10']['success_rate']:>6.1f}% {results['restart10']['time']:>14.3f} {results['restart10']['avg_restarts']:>14.1f}")
    print(f"Random Restart (k=100){' '*3} {results['restart100']['success_rate']:>6.1f}% {results['restart100']['time']:>14.3f} {results['restart100']['avg_restarts']:>14.1f}")
    print(f"Stochastic Hill Climbing{' '*2} {results['stochastic']['success_rate']:>6.1f}% {results['stochastic']['time']:>14.3f} {'-':>14}")
    print(f"Simulated Annealing{' '*7} {results['simulated_annealing']['success_rate']:>6.1f}% {results['simulated_annealing']['time']:>14.3f} {'-':>14}")

    return results

if __name__ == "__main__":
    results = run_experiments()


Running experiments with N=20, Trials=100

Testing Basic Hill Climbing...
Completed 10/100 trials...
Completed 20/100 trials...
Completed 30/100 trials...
Completed 40/100 trials...
Completed 50/100 trials...
Completed 60/100 trials...
Completed 70/100 trials...
Completed 80/100 trials...
Completed 90/100 trials...
Completed 100/100 trials...

Testing Random Restart (k=10)...
Completed 10/100 trials...
Completed 20/100 trials...
Completed 30/100 trials...
Completed 40/100 trials...
Completed 50/100 trials...
Completed 60/100 trials...
Completed 70/100 trials...
Completed 80/100 trials...
Completed 90/100 trials...
Completed 100/100 trials...

Testing Random Restart (k=100)...
Completed 10/100 trials...
Completed 20/100 trials...
Completed 30/100 trials...
Completed 40/100 trials...
Completed 50/100 trials...
Completed 60/100 trials...
Completed 70/100 trials...
Completed 80/100 trials...
Completed 90/100 trials...
Completed 100/100 trials...

Testing Stochastic Hill Climbing...
Comple